# Segmenting and clustering neighbourhoods in Toronto

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## GET request for url

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Server': 'mw1244.eqiad.wmnet', 'X-Varnish': '287554132 965142833, 606084687 172077033', 'X-Analytics': 'ns=0;page_id=539066;https=1;nocookies=1', 'X-Client-IP': '158.175.98.172', 'X-Cache': 'cp1081 hit/9, cp1081 hit/24', 'Accept-Ranges': 'bytes', 'X-Powered-By': 'HHVM/3.18.6-dev', 'Backend-Timing': 'D=166170 t=1561110002096988', 'Content-language': 'en', 'Content-Length': '14904', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate', 'Set-Cookie': 'WMF-Last-Access=22-Jun-2019;Path=/;HttpOnly;secure;Expires=Wed, 24 Jul 2019 00:00:00 GMT, WMF-Last-Access-Global=22-Jun-2019;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Wed, 24 Jul 2019 00:00:00 GMT, GeoIP=US:::37.75:-97.82:v4; Path=/; secure; Domain=.wikipedia.org', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'D

### Creating a BeautifulSoup object

In [4]:
soup=BeautifulSoup(result.content,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":900271985,"wgRevisionId":900271985,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

### Parsing the table we want to convert into python dataframe

In [5]:
table=soup.find('table',{'class':'wikitable sortable'})
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

## Converting the content of the table into a list

In [6]:
trs=table.find_all('tr')
rows=[]
for tr in trs:
    i=tr.find_all('td')
    if i:
        rows.append(i)
list=[]
for row in rows:
    postalcode=row[0].text.rstrip()
    borough=row[1].text.rstrip()
    neighborhood=row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        list.append([postalcode, borough, neighborhood])
list


[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", "Queen's Park"],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Etobicoke', 'Martin Grove'],
 ['M9B', 'Etobicoke', 'Princess Gardens'],
 ['M9B', 'Etobicoke', 'West Deane Park'],
 ['M1C', 'Scarborough', 'Highland Creek'],
 ['M1C', 'Scarborough', 'Rouge Hill'],
 ['M1C', 'Scarborough', 'Port Union'],
 ['M3C', 'North 

### Creating the data frame and displaying the initial shape

In [7]:
columns=['Postal Code','Borough', 'Neighborhood']
df=pd.DataFrame(list,columns=columns)
print(df.shape)

(211, 3)


In [8]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


# Getting longitudes and latitudes

In [9]:
!pip install wikipedia

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [10]:
import wikipedia as wp

In [11]:
import pandas as pd
import numpy as np

In [12]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-06-22 10:37:02--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-06-22 10:37:02--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-22 10:37:05--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-22 10:37:05--  https://ibm.box.com/public/static/9afzr83pps4pwf2

In [13]:
df_cord=pd.read_csv('GeoCord.csv')

In [14]:
df = pd.merge(df, df_cord, on=['Postal Code'], how='inner')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [15]:
CLIENT_ID = 'Y2PZEH32X5SCASEI0LRNA1FDWSCN5SPFS0VAXWKQ0OHJ34AT' # your Foursquare ID
CLIENT_SECRET = 'LS0YEOSIW5WHMUFIFGVAQLXBL1QQ3TF5YL51W2HR5NGVH35Y' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y2PZEH32X5SCASEI0LRNA1FDWSCN5SPFS0VAXWKQ0OHJ34AT
CLIENT_SECRET:LS0YEOSIW5WHMUFIFGVAQLXBL1QQ3TF5YL51W2HR5NGVH35Y


# Exploring neighbourhoods

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Exploring toronto neighbourhoods using Foursquare API

In [28]:
def getNearbyVenues(names, boro, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, boro, lat, lng in zip(names, boro, latitudes, longitudes):
        print("Fetching venues for : ",name)            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            boro,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Borough',
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)
   

In [29]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude'],boro=df['Borough'])

Fetching venues for :  Parkwoods
Fetching venues for :  Victoria Village
Fetching venues for :  Harbourfront
Fetching venues for :  Regent Park
Fetching venues for :  Lawrence Heights
Fetching venues for :  Lawrence Manor
Fetching venues for :  Queen's Park
Fetching venues for :  Islington Avenue
Fetching venues for :  Rouge
Fetching venues for :  Malvern
Fetching venues for :  Don Mills North
Fetching venues for :  Woodbine Gardens
Fetching venues for :  Parkview Hill
Fetching venues for :  Ryerson
Fetching venues for :  Garden District
Fetching venues for :  Glencairn
Fetching venues for :  Cloverdale
Fetching venues for :  Islington
Fetching venues for :  Martin Grove
Fetching venues for :  Princess Gardens
Fetching venues for :  West Deane Park
Fetching venues for :  Highland Creek
Fetching venues for :  Rouge Hill
Fetching venues for :  Port Union
Fetching venues for :  Flemingdon Park
Fetching venues for :  Don Mills South
Fetching venues for :  Woodbine Heights
Fetching venues f

In [30]:
toronto_venues.shape[0]

2668

In [31]:
toronto_venues.head()

,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,North York,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,North York,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,North York,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,North York,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,North York,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### Counting the type of venues available in different neighbourhoods

In [32]:
toronto_venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False).head(10)

Venue Category
Coffee Shop             178
Café                    130
Pizza Place              90
Park                     73
Sandwich Place           67
Bakery                   61
Restaurant               60
Fast Food Restaurant     54
Italian Restaurant       52
Bar                      49
Name: Neighborhood, dtype: int64

# Getting number of venues per Neighborhood

In [33]:
individual_toronto_venue_count = toronto_venues.groupby(['Borough','Neighborhood'])['Borough'].count().to_frame()

In [34]:
individual_toronto_venue_count

Borough
Borough          Neighborhood                                              
Central Toronto  Davisville                                              30
                 Davisville North                                         9
                 Deer Park                                               14
                 Forest Hill North                                        4
                 Forest Hill SE                                          14
                 Forest Hill West                                         4
                 Lawrence Park                                            3
                 Moore Park                                               2
                 North Midtown                                           23
                 North Toronto West                                      16
                 Rathnelly                                               14
                 Roselawn                                                 2
                 South Hill                                              14
                 Summerhill East                                          2
                 Summerhill West                                         14
                 The Annex                                               23
                 Yorkville                                               23
Downtown Toronto Adelaide                                                30
                 Bathurst Quay                                           16
                 Berczy Park                                             30
                 CN Tower                                                16
                 Cabbagetown                                             30
                 Central Bay Street                                      30
                 Chinatown                                               30
                 Christie                                                15
                 Church and Wellesley                                    30
                 Commerce Court                                          30
                 Design Exchange                                         30
                 First Canadian Place                                    30
                 Garden District                                         30
                 Grange Park                                             30
                 Harbord                                                 30
                 Harbourfront                                            30
                 Harbourfront East                                       30
                 Harbourfront West                                       16
                 Island airport                                          16
                 Kensington Market                                       30
                 King                                                    30
                 King and Spadina                                        16
                 Railway Lands                                           16
                 Regent Park                                             30
                 Richmond                                                30
                 Rosedale                                                 4
                 Ryerson                                                 30
                 South Niagara                                           16
                 St. James Town                                          60
                 Stn A PO Boxes 25 The Esplanade                         30
                 Toronto Dominion Centre                                 30
                 Toronto Islands                                         30
                 Underground city                                        30
                 Union Station                                           30
                 University of Toronto  

In [35]:
individual_toronto_venue_count.rename(columns={'Borough':'Number of venues'},inplace=True)

In [36]:
individual_toronto_venue_count.reset_index(inplace=True)

In [37]:
individual_toronto_venue_count

,Borough,Neighborhood,Number of venues
0,Central Toronto,Davisville,30
1,Central Toronto,Davisville North,9
2,Central Toronto,Deer Park,14
3,Central Toronto,Forest Hill North,4
4,Central Toronto,Forest Hill SE,14
5,Central Toronto,Forest Hill West,4
6,Central Toronto,Lawrence Park,3
7,Central Toronto,Moore Park,2
8,Central Toronto,North Midtown,23
9,Central Toronto,North Toronto West,16


# One hot encoding for letting us compare venues

In [38]:
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']])

In [39]:
toronto_venues_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_venues_grouped = toronto_venues_onehot.groupby('Neighborhood').mean().reset_index()
toronto_venues_grouped

,Neighborhood,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,Venue Category_Art Gallery,Venue Category_Art Museum,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Garage,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Basketball Court,Venue Category_Basketball Stadium,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Belgian Restaurant,Venue Category_Bike Shop,Venue Category_Bistro,Venue Category_Boat or Ferry,Venue Category_Bookstore,Venue Category_Boutique,Venue Category_Brazilian Restaurant,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bridal Shop,Venue Category_Bubble Tea Shop,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Bus Stop,Venue Category_Butcher,Venue Category_Cafeteria,Venue Category_Café,Venue Category_Cajun / Creole Restaurant,Venue Category_Candy Store,Venue Category_Caribbean Restaurant,Venue Category_Check Cashing Service,Venue Category_Cheese Shop,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Church,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_College Arts Building,Venue Category_College Gym,Venue Category_College Stadium,Venue Category_Comfort Food Restaurant,Venue Category_Comic Shop,Venue Category_Concert Hall,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Costume Shop,Venue Category_Coworking Space,Venue Category_Creperie,Venue Category_Cuban Restaurant,Venue Category_Cupcake Shop,Venue Category_Curling Ice,Venue Category_Dance Studio,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Dog Run,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Empanada Restaurant,Venue Category_Ethiopian Restaurant,Venue Category_Event Space,Venue Category_Falafel Restaurant,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Field,Venue Category_Fish & Chips Shop,Venue Category_Fish Market,Venue Category_Flea Market,Venue Category_Flower Shop,Venue Category_Food,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_Food Truck,Venue Category_Fountain,Venue Category_Fraternity House,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Fruit & Vegetable Store,Venue Category_Furniture / Home Store,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gastropub,Venue Category_Gay Bar,Venue Category_General Entertainment,Venue Category_Gift Shop,Venue Category_Gluten-free Restaurant,Venue Category_Golf Course,Venue Category_Gourmet Shop,Venue Category_Greek Restaurant,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Venue Category_Hakka Restaurant,Venue Category_Harbor / Marina,Venue Category_Health Food Store,Venue Category_Historic Site,Venue Category_History Museum,Venue Category_Hobby Shop,Venue Category_Hockey Arena,Venue Category_Home Service,Venue Category_Hostel,Venue Category_Hotel,Venue Category_Housing Development,Venue Category_Ice Cream Shop,Venue Category_Indian Restaurant,Venue Category_Indie Movie Theater,Venue Categ

In [40]:

number_of_top_venues = 5

In [41]:
for hood in toronto_venues_grouped['Neighborhood']:
    print('---------',hood,'---------')
    temp = toronto_venues_grouped[toronto_venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(number_of_top_venues))
    print('\n')

--------- Adelaide ---------
                             Venue  Frequency
0              Venue Category_Café       0.10
1        Venue Category_Steakhouse       0.10
2       Venue Category_Pizza Place       0.07
3             Venue Category_Hotel       0.07
4  Venue Category_Asian Restaurant       0.07


--------- Agincourt ---------
                               Venue  Frequency
0              Venue Category_Lounge       0.25
1      Venue Category_Breakfast Spot       0.25
2  Venue Category_Chinese Restaurant       0.25
3      Venue Category_Sandwich Place       0.25
4   Venue Category_Accessories Store       0.00


--------- Agincourt North ---------
                              Venue  Frequency
0         Venue Category_Playground       0.33
1   Venue Category_Asian Restaurant       0.33
2               Venue Category_Park       0.33
3  Venue Category_Accessories Store       0.00
4  Venue Category_Mobile Phone Shop       0.00


--------- Albion Gardens ---------
                  

In [42]:
def return_most_common_venues(row, number_of_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:number_of_top_venues]

### Extracting 

In [44]:
number_of_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(number_of_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_venues_grouped['Neighborhood']

for ind in np.arange(toronto_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_venues_grouped.iloc[ind, :], number_of_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Venue Category_Steakhouse,Venue Category_Café,Venue Category_Hotel,Venue Category_Pizza Place,Venue Category_Asian Restaurant,Venue Category_Noodle House,Venue Category_Food Court,Venue Category_Seafood Restaurant,Venue Category_Smoke Shop,Venue Category_Lounge
1,Agincourt,Venue Category_Lounge,Venue Category_Breakfast Spot,Venue Category_Chinese Restaurant,Venue Category_Sandwich Place,Venue Category_Dance Studio,Venue Category_Electronics Store,Venue Category_Eastern European Restaurant,Venue Category_Dumpling Restaurant,Venue Category_Drugstore,Venue Category_Dog Run
2,Agincourt North,Venue Category_Asian Restaurant,Venue Category_Park,Venue Category_Playground,Venue Category_Curling Ice,Venue Category_Electronics Store,Venue Category_Eastern European Restaurant,Venue Category_Dumpling Restaurant,Venue Category_Drugstore,Venue Category_Dog Run,Venue Category_Discount Store
3,Albion Gardens,Venue Category_Grocery Store,Venue Category_Fried Chicken Joint,Venue Category_Coffee Shop,Venue Category_Beer Store,Venue Category_Fast Food Restaurant,Venue Category_Sandwich Place,Venue Category_Liquor Store,Venue Category_Pharmacy,Venue Category_Video Store,Venue Category_Pizza Place
4,Alderwood,Venue Category_Pizza Place,Venue Category_Coffee Shop,Venue Category_Pool,Venue Category_Dance Studio,Venue Category_Gym,Venue Category_Pharmacy,Venue Category_Skating Rink,Venue Category_Pub,Venue Category_Sandwich Place,Venue Category_Curling Ice
